In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# NVIDIA Tesla T4
# ARCH= -gencode arch=compute_75

# NVIDIA Tesla P4
# ARCH= -gencode arch=compute_61

# GP100/Tesla P100
# ARCH= -gencode arch=compute_60

# don't touch anything if you use NVIDIA Tesla K80

%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# depending on the GPU Compute Capability you are using you must change some lines of Makefile
!sed -i 's/# ARCH= -gencode arch=compute_75/ARCH= -gencode arch=compute_75/' Makefile

# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# connect with your Google Drive account
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

In [ ]:
# this is where my datasets are stored within my Google Drive 
!ls /mydrive/yolov4-mask

In [8]:
!cp /mydrive/yolov4-mask/obj.zip ../
!cp /mydrive/yolov4-mask/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in ./darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [10]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov4-mask/yolov4-mask.cfg ./cfg

In [11]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4-mask/mask.names ./data
!cp /mydrive/yolov4-mask/mask.data  ./data

In [12]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov4-mask/generate_train.py ./
!cp /mydrive/yolov4-mask/generate_test.py ./

In [13]:
# generate train.txt and test.txt files
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

**CRTL + SHIFT + i** ---> Copy the following into the console and execute to avoid detecting inactivity on the Notebook
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/mask.data cfg/yolov4-mask.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/mask.data cfg/yolov4-mask.cfg /mydrive/yolov4-mask/backup/yolov4-mask_last.weights -dont_show -map

In [ ]:
!./darknet detector map data/mask.data cfg/yolov4-mask.cfg /mydrive/yolov4-mask/backup/yolov4-mask_best.weights
#!./darknet detector map data/mask.data cfg/yolov4-mask.cfg /mydrive/yolov4-mask/backup/yolov4-mask_last.weights